In [7]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage,ToolMessage
from langchain_core.tools import tool
from langchain_core.tools.structured import StructuredTool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
import os
from rich import print

load_dotenv()

True

In [8]:
import random


@tool
def get_weather(location: str) -> str:
    """
    Retrieve the current weather for a specified location.

    Args:
        location (str): The name of the location for which to retrieve weather information.

    Returns:
        str: A brief description of the current weather conditions at the specified location.
    """
    if location.lower() in ["sf", "san francisco"]:
        return "The weather is cool and foggy with a mild temperature."
    else:
        return "The weather is warm and sunny with clear skies."


@tool
def get_coolest_cities() -> str:
    """
    Retrieve a list of cities that are considered cool or popular.

    Returns:
        str: A comma-separated list of popular cities known for their unique appeal.
    """
    return "New York City, San Francisco, Tokyo, Paris, Berlin"


@tool
def get_mom_status(query: str) -> str:
    """
    Respond to inquiries about mom's status or availability.

    Args:
        query (str): A specific question or context related to mom.

    Returns:
        str: A status update or response based on the inquiry.
    """
    return random.choice(["Available", "Unavailable", "Busy", "Free", "Not reachable"])


@tool
def get_dad_status(query: str) -> str:
    """
    Respond to inquiries about dad's status or availability.

    Args:
        query (str): A specific question or context related to dad.

    Returns:
        str: A status update or response based on the inquiry.
    """
    return random.choice(["Available", "Unavailable", "Busy", "Free", "Not reachable"])


@tool
def get_github_project_info(query: str) -> str:
    """
    Retrieve information about projects from a GitHub repository.

    Args:
        query (str): A keyword or description of the type of project information requested.

    Returns:
        str: A brief summary of GitHub projects associated with the user or query.
    """
    return "Your GitHub portfolio includes projects such as a proxy checker, a Telegram views booster, and a reaction enhancer."


@tool
def get_local_project_info(query: str) -> str:
    """
    Retrieve information about projects currently being developed locally.

    Args:
        query (str): A keyword or description of the project information being requested.

    Returns:
        str: A summary of active projects or ongoing developments on the local system.
    """
    return "Currently, you are working on building an autonomous agent for web crawling and data extraction."

In [9]:
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [10]:
from langchain_core.messages.tool import ToolCall
from google.generativeai.types import Tool
from typing import Dict

def callTools(tool_calls: ToolCall,AvailableTools:Dict[str,StructuredTool]):
    toolCallResult = list()
    for tool_call in tool_calls:
        selected_tool = AvailableTools[tool_call["name"].lower()]
        print(f"Calling -----> {selected_tool.name}")
        tool_msg:ToolMessage = selected_tool.invoke(tool_call)
        toolCallResult.append(tool_msg)
    print(toolCallResult)
    return toolCallResult

In [11]:
@tool
def handle_weather_query(query: str) -> str:
    """
    Handle weather-related queries, such as current weather, temperature trends, or weather conditions
    in a specific location.

    Args:
        query (str): A description of the weather-related query (e.g., "current weather", "forecast").

    Returns:
        str: A response based on the weather-related query and location provided.
    """
    WeatherServiceAgent = model.bind_tools([get_weather, get_coolest_cities])
    raise ValueError("Under Development")


@tool
def handle_family_query(query: str) -> str:
    """
    Handle queries related to family members, such as their availability, status, or any other information. It can handle multiple family members at a time.

    Args:
        query (str): A description of the query related to family members (e.g., "mom's status", "dad's availability", or "are mom and dad available?").

    Returns:
        str: A response based on the family-related query.
    """

    
    local_msgs=[
        SystemMessage(content="You are an assistant tasked with handling queries related to Roshan Yadav's family members, including their availability, status, and any other relevant information."),
        HumanMessage(content=query)
    ]
    tools_func = [get_dad_status, get_mom_status]
    AvailableTools = {tool.name:tool for tool in tools_func}

    FamilyAgent = ChatGoogleGenerativeAI(model="gemini-1.5-pro").bind_tools(tools_func)
    # FamilyAgent = ChatOllama(model="llama3-groq-tool-use:latest", temperature=0).bind_tools(tools_func)
    
    FamilyAgentResponse:AIMessage = FamilyAgent.invoke(local_msgs)
    local_msgs.append(FamilyAgentResponse)
    if FamilyAgentResponse.tool_calls:
        toolmessages = callTools(FamilyAgentResponse.tool_calls,AvailableTools)
        local_msgs+=toolmessages
        print(local_msgs)
        FamilyAgentResponse:AIMessage = FamilyAgent.invoke(local_msgs)
        print(FamilyAgentResponse)
    return FamilyAgentResponse.content


@tool
def handle_project_query(query: str) -> str:
    """
    Handle queries related to projects, such as active projects, completed ones, or those stored locally or on GitHub.

    Args:
        query (str): A description of the query about projects (e.g., "What projects am I working on?", "Show my GitHub projects").

    Returns:
        str: A response with information about your projects based on the query.
    """
    ProjectsAgent = model.bind_tools([get_github_project_info, get_local_project_info])
    raise ValueError("Under Development")
    

In [12]:
tools_func = [handle_weather_query, handle_family_query, handle_project_query]
AvailableTools = {tool.name:tool for tool in tools_func}
PersonalAssistant = model.bind_tools(tools_func)

wrapper_msgs = [
    SystemMessage(content="You are a personal assistant for Roshan Yadav, aware of all his professional and personal details, and your role is to assist with tasks, offer advice, and provide solutions based on his goals, preferences, and ongoing projects."),
    HumanMessage(content="Are my parents available this evening? If they are, we can plan dinner. Could you please check and confirm if it's possible?")
]
aimsg = PersonalAssistant.invoke(wrapper_msgs)
wrapper_msgs.append(aimsg)
if aimsg.tool_calls:
    wrapper_msgs += callTools(aimsg.tool_calls,AvailableTools)
    aimsg: AIMessage = PersonalAssistant.invoke(wrapper_msgs)
    wrapper_msgs.append(aimsg)
print(wrapper_msgs)

Calling -----> handle_family_query

Calling -----> get_dad_status

Calling -----> get_mom_status

[
    ToolMessage(content='Free', name='get_dad_status', tool_call_id='e537fb1a-f38c-49c1-a509-9e95b995d12c'),
    ToolMessage(content='Available', name='get_mom_status', tool_call_id='5ca9b97d-8349-443d-b130-b574b57b4220')
]

[
    SystemMessage(
        content="You are an assistant tasked with handling queries related to Roshan Yadav's family members, 
including their availability, status, and any other relevant information.",
        additional_kwargs={},
        response_metadata={}
    ),
    HumanMessage(content='Are mom and dad available this evening?', additional_kwargs={}, response_metadata={}),
    AIMessage(
        content='',
        additional_kwargs={
            'function_call': {
                'name': 'get_mom_status',
                'arguments': '{"query": "Is mom available this evening?"}'
            }
        },
        response_metadata={
            'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
            'finish_reason': 'STOP',
            'safety_ratings': []
        },
        id='run-d64f6d3e-df9d-4da0-8dee-31eb2b657d0e-0',
        tool_calls=[
            {
                'name': 'get_dad_status',
                'args': {'query': 'Is dad available this evening?'},
                'id': 'e537fb1a-f38c-49c1-a509-9e95b995d12c',
                'type': 'tool_call'
            },
            {
                'name': 'get_mom_status',
                'args': {'query': 'Is mom available this evening?'},
                'id': '5ca9b97d-8349-443d-b130-b574b57b4220',
                'type': 'tool_call'
            }
        ],
        usage_metadata={
            'input_tokens': 199,
            'output_tokens': 24,
            'total_tokens': 223,
            'input_token_details': {'cache_read': 0}
        }
    ),
    ToolMessage(content='Free', name='get_dad_status', tool_call_id='e537fb1a-f38c-49c1-a509-9e95b995d12c'),
    ToolMessage(content='Available', name='get_mom_status', tool_call_id='5ca9b97d-8349-443d-b130-b574b57b4220')
]

AIMessage(
    content='Yes, both Mom and Dad are available this evening. Dad is free and Mom is available.\n',
    additional_kwargs={},
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'safety_ratings': []
    },
    id='run-60ebb436-513c-4797-a1d5-406d32307620-0',
    usage_metadata={
        'input_tokens': 280,
        'output_tokens': 20,
        'total_tokens': 300,
        'input_token_details': {'cache_read': 0}
    }
)

[
    ToolMessage(
        content='Yes, both Mom and Dad are available this evening. Dad is free and Mom is available.\n',
        name='handle_family_query',
        tool_call_id='ac9aa990-ac52-4295-a103-f729628c6610'
    )
]

[
    SystemMessage(
        content='You are a personal assistant for Roshan Yadav, aware of all his professional and personal details,
and your role is to assist with tasks, offer advice, and provide solutions based on his goals, preferences, and 
ongoing projects.',
        additional_kwargs={},
        response_metadata={}
    ),
    HumanMessage(
        content="Are my parents available this evening? If they are, we can plan dinner. Could you please check and
confirm if it's possible?",
        additional_kwargs={},
        response_metadata={}
    ),
    AIMessage(
        content='',
        additional_kwargs={
            'function_call': {
                'name': 'handle_family_query',
                'arguments': '{"query": "Are mom and dad available this evening?"}'
            }
        },
        response_metadata={
            'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
            'finish_reason': 'STOP',
            'safety_ratings': []
        },
        id='run-d2a6c612-af90-40ad-915f-5324654efc72-0',
        tool_calls=[
            {
                'name': 'handle_family_query',
                'args': {'query': 'Are mom and dad available this evening?'},
                'id': 'ac9aa990-ac52-4295-a103-f729628c6610',
                'type': 'tool_call'
            }
        ],
        usage_metadata={
            'input_tokens': 414,
            'output_tokens': 14,
            'total_tokens': 428,
            'input_token_details': {'cache_read': 0}
        }
    ),
    ToolMessage(
        content='Yes, both Mom and Dad are available this evening. Dad is free and Mom is available.\n',
        name='handle_family_query',
        tool_call_id='ac9aa990-ac52-4295-a103-f729628c6610'
    ),
    AIMessage(
        content='Yes, both Mom and Dad are available this evening.  We can plan dinner.\n',
        additional_kwargs={},
        response_metadata={
            'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
            'finish_reason': 'STOP',
            'safety_ratings': []
        },
        id='run-23dc555b-8869-47e7-8d18-80bd0a3cf256-0',
        usage_metadata={
            'input_tokens': 476,
            'output_tokens': 18,
            'total_tokens': 494,
            'input_token_details': {'cache_read': 0}
        }
    )
]